In [ ]:
!pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jalilahmadafshar","key":"740cf1508dcb795db03b2d4ae80f16f6"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset
!unzip fake-and-real-news-dataset.zip

Dataset URL: https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset
License(s): CC-BY-NC-SA-4.0
fake-and-real-news-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  fake-and-real-news-dataset.zip
  inflating: Fake.csv                
  inflating: True.csv                


In [ ]:
import pandas as pd


In [ ]:
fake = pd.read_csv("Fake.csv")
true = pd.read_csv("True.csv")

fake["label"] = 1  # Fake = 1
true["label"] = 0  # True = 0

In [ ]:
df = pd.concat([fake, true]).sample(frac=1).reset_index(drop=True)
print("✅ Data loaded successfully!")
print(df.head())


✅ Data loaded successfully!
                                               title  \
0  UK government official says 'does not recogniz...   
1  Japan defends Toyota after Trump broadside ove...   
2  Russian communists drop veteran, select surpri...   
3  Trump Rally In Austin TX – Protesters Largely ...   
4  Israel strikes Hamas targets in Gaza after Pal...   

                                                text       subject  \
0  LONDON (Reuters) - A British government offici...     worldnews   
1  TOKYO (Reuters) - The Japanese government defe...  politicsNews   
2  MOSCOW (Reuters) - The Russian Communist Party...     worldnews   
3  21st Century Wire says Trump supporters gather...   Middle-east   
4  JERUSALEM (Reuters) - The Israeli military sai...     worldnews   

                 date  label  
0  November 28, 2017       0  
1    January 6, 2017       0  
2  December 23, 2017       0  
3       March 5, 2017      1  
4  December 18, 2017       0  


In [ ]:
df["content"] = df["title"].astype(str) + " " + df["text"].astype(str)
df = df[["content", "label"]]

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [ ]:
nltk.download("stopwords")
ps = PorterStemmer()
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def clean_text(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    text = text.lower().split()
    text = [ps.stem(w) for w in text if w not in stop_words]
    return " ".join(text)

df["cleaned"] = df["content"].apply(clean_text)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df["cleaned"], df["label"], test_size=0.2, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

LogisticRegression()

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = model.predict(X_test_tfidf)
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

✅ Accuracy: 0.9888641425389755

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      4332
           1       0.99      0.99      0.99      4648

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980


Confusion Matrix:
 [[4296   36]
 [  64 4584]]


In [ ]:
sample = "The president announced new reforms to boost the economy."
cleaned = clean_text(sample)
vectorized = vectorizer.transform([cleaned])
pred = model.predict(vectorized)

print(f"\n🧠 Prediction: {'FAKE' if pred[0]==1 else 'REAL'}")


🧠 Prediction: FAKE
